# Introduction

The following uses a kind of literate programming approach to build a library of tools useful for writing unit and integration tests directly into a notebook. The library is to be articulated as a Python package built as the concatenation of a subset of the code cells of this notebook, using an ad hoc script. To help with identifying which code cells are parts of the final package and which are inline testing code, we use *tags*, which make up cell metadata in this notebook.

In [58]:
from abc import ABC, abstractmethod
from contextlib import contextmanager, ExitStack
from copy import copy, deepcopy
from io import RawIOBase
import sys
from traceback import extract_tb, StackSummary
from typing import ContextManager, Dict, List, Tuple, Iterator, Union, Iterable, Optional, Any

# Test results

In [67]:
class Result(ABC):
    """
    Result of a test. Indicates whether the test passed (was a success), and if it did not,
    whether it was a failure (as opposed to any other kind of issue).
    """
    
    @abstractmethod
    def is_success(self) -> bool:
        """True when an associated test run has passed."""
        raise NotImplementedError()

    def is_failure(self) -> bool:
        """True when an associated has not passed because a designed failure condition was met."""
        return False
    
    def as_dict(self) -> Dict:
        """Expresses this result as a dictionary suitable to structured data serialization."""
        return {"type": type(self).__name__}

## Success

In [52]:
class Success(Result):
    """
    Result for a test that passed.
    """
    def is_success(self) -> bool:
        return True

In [53]:
assert Success().is_success()

In [54]:
assert Success().as_dict() == {"type": "Success"}

## Error

In [85]:
class Error(Result):
    """Non-passing test result due to an exception being raised."""
    
    def __init__(self) -> None:
        super().__init__()
        self._type_exc: type
        self._value_exc: Any
        self._type_exc, self._value_exc, tb = sys.exc_info()
        if tb is None:
            raise RuntimeError("Can only instantiate this class when an exception has been raised.")
        self._traceback: StackSummary = extract_tb(tb)
        
    def is_success(self) -> bool:
        return False
    
    @property
    def type_exc(self) -> type:
        """Returns the type of the exception associated to this result."""
        return self._type_exc
    
    @property
    def value_exc(self) -> Any:
        """Returns the exception raised in association to this test result."""
        return self._value_exc
    
    @property
    def traceback(self) -> StackSummary:
        """
        Returns a summary of the stack trace associated to the exception that brought this test result.
        """
        return self._traceback
    
    def as_dict(self) -> Dict:
        d = super().as_dict()
        d.update(
            {
                "type_exc": self.type_exc.__name__,
                "value_exc": str(self.value_exc),
                "traceback": [
                    {
                        "filename": filename,
                        "lineno": lineno,
                        "context": context,
                        "line_code": line_code
                    }
                    for filename, lineno, context, line_code in [tuple(frame) for frame in self.traceback]
                ]
            }
        )
        return d

In [86]:
try:
    raise RuntimeError()
except:
    err: Error = Error()
    assert not err.is_success()
    assert not err.is_failure()
    assert err.type_exc == RuntimeError
    assert isinstance(err.value_exc, RuntimeError)
    assert isinstance(err.traceback, StackSummary)

In [112]:
# This trick gets us a cell's "file name", given that the `__file__` constant is not defined
# in Jupyter notebooks.
import inspect
def _asdf():
    pass
filename = inspect.getfile(_asdf)

try:
    raise RuntimeError()
except:
    assert {
        "type": "Error",
        "type_exc": "RuntimeError",
        "value_exc": "",
        "traceback": [
            {
                "filename": filename,
                "lineno": 9,
                "context": "<module>",
                "line_code": "raise RuntimeError()"
            }
        ]
    } == Error().as_dict()

## Failure

For convenience's sake, we model `Failure`s as a subclass of `Error` to gain the exception breakdown functionality.

In [110]:
class Failure(Error):
    """
    Test result stemming from a condition check that failed, or a test run marked
    as a failure.
    """
    def __init__(self, reason: str):
        super().__init__()
        self._reason = reason
        
    @property
    def reason(self) -> str:
        "Reason given by the programmer as to why the test failed."
        return self._reason
    
    def is_failure(self) -> bool:
        return True
    
    def as_dict(self) -> Dict:
        d = super().as_dict()
        d["reason"] = self.reason
        return d

In [113]:
try:
    assert False
except:
    err: Failure = Failure("asdf")
    assert not err.is_success()
    assert err.is_failure()
    assert err.type_exc == AssertionError
    assert isinstance(err.value_exc, AssertionError)
    assert isinstance(err.traceback, StackSummary)

In [117]:
import inspect
def _asdf():
    pass
filename = inspect.getfile(_asdf)

try:
    assert False
except:
    assert {
        "type": "Failure",
        "type_exc": "AssertionError",
        "value_exc": "",
        "traceback": [
            {
                "filename": filename,
                "lineno": 7,
                "context": "<module>",
                "line_code": "assert False"
            }
        ],
        "reason": "asdf"
    } == Failure("asdf").as_dict()

# Individual tests

In [9]:
class TestFailed(Exception):
    """
    Exception raised by this framework in order to mark a test run as a Failure.
    """
    
    def __init__(self, reason: str) -> None:
        super().__init__(reason)
        self.reason = reason

In [118]:
try:
    raise TestFailed("asdf")
except TestFailed as err:
    assert str(err) == "asdf"

In [10]:
class Test:
    """
    Object passed to a test code fragment, so it can communication test run outcomes
    to the test framework.
    """
    def fail(self, reason: str = ""):
        "Marks the ongoing test as failed."
        raise TestFailed(reason)

In [11]:
try:
    Test().fail("asdf")
    assert False
except TestFailed as err:
    assert err.reason == "asdf"

# Environment protection

In [12]:
@contextmanager
def protect_environment(*names: str) -> ContextManager:
    """
    Isolates the notebook's environment (variables) from redefinition and the definition
    of new symbols during execution of the context. In addition, any variable named in
    parameter is protected from any state change during execution of the context.
    """
    assert get_ipython().ns_table["user_local"] is get_ipython().ns_table["user_global"]

    namespace_orig = copy(globals())
    for name in names:
        if name in namespace_orig:
            namespace_orig[name] = deepcopy(namespace_orig[name])
    
    try:
        yield
    finally:
        G = globals()
        G.clear()
        G.update(namespace_orig)
        for field in ["user_global", "user_local"]:
            get_ipython().ns_table[field] = namespace_orig

In [13]:
mylist = [1, 2, 3, 4, 5]
assert "otherlist" not in get_ipython().ns_table["user_local"]

with protect_environment():
    otherlist = [10, 11, 12]
    assert len(otherlist) == 3
    mylist.pop()
    mylist = [90]
    
assert "otherlist" not in get_ipython().ns_table["user_local"]
assert mylist == [1, 2, 3, 4]

In [14]:
mylist = [1, 2, 3, 4, 5]
assert "otherlist" not in get_ipython().ns_table["user_local"]

with protect_environment("mylist"):
    otherlist = [10, 11, 12]
    assert len(otherlist) == 3
    mylist.pop()
    
assert "otherlist" not in get_ipython().ns_table["user_local"]
assert mylist == [1, 2, 3, 4, 5]

# Test suites

In [123]:
class Suite:
    """
    Suite of tests, gathering the result of multiple named test runs. Test code fragments
    are named using the `test()` context manager.
    """
    
    def __init__(self) -> None:
        self._tests: Dict[str, List[Result]] = {}

    @contextmanager
    def test(self, name: str, protect_env: Union[bool, Iterable[str]] = True) -> ContextManager[Test]:
        """
        Starts a named testing code fragment. The fragment is run right away, which produces
        a certain test Result that is retained by the Suite instance.
        
        name        - Name of the test
        protect_env - If set to True, any symbol defined or redefined by the code in context
                      of this manager is undone when popping out of the context. This facilitates
                      test isolation. If, instead of True, an iterable sequence of names is passed
                      as value to this parameter, the objects corresponding to these names in the
                      user's namespace are saved by deep copy, thereby protecting these objects
                      from any state change as well. If False is given as parameter value, the
                      user's environment is not isolated from the test code, making any any definition
                      or state change definitive (which is the usual behaviour when computing with
                      notebooks).
        """
        with ExitStack() as stack:
            if protect_env is not False:
                stack.enter_context(
                    protect_environment(*(protect_env if hasattr(protect_env, "__iter__") else []))
                )
            append_result = self._tests.setdefault(name, []).append
            try:
                yield Test()
                result_args = ()
                append_result(Success())
            except TestFailed as err:
                append_result(Failure(err.reason or "test marked as failed"))
            except AssertionError as err:
                append_result(Failure(str(err) or "assertion failed"))
            except BaseException:
                append_result(Error())
            
    @property
    def results(self) -> Iterator[Tuple[str, Iterator[Result]]]:
        """
        Iterates through the gathered test results. For each named test, yields a tuple of
        the name of the test and an iterator over each result gathered as the test has
        been run.
        """
        for name, test_results in self._tests.items():
            yield name, iter(test_results)
            
    def as_dict(self) -> Dict[str, List[Dict]]:
        "Provides a structured data representation suitable for data serialization and exportation."
        return {name: [r.as_dict() for r in rez] for name, rez in self.results}

In [16]:
assert isinstance(Suite()._tests, dict)

In [17]:
with Suite().test("sanity-check") as test:
    assert test is not None

In [18]:
suite = Suite()

with suite.test("succeeding"):
    assert True
    
with suite.test("failing-by-assert-terse"):
    assert False
    
with suite.test("failing-by-assert-reason"):
    assert False, "assert reason"
    
with suite.test("failing-manually-terse") as test:
    test.fail()
    
with suite.test("failing-manually-reason") as test:
    test.fail("fail reason")
    
with suite.test("error"):
    raise RuntimeError("doh")

assert [
    ("succeeding", [(Success, "")]),
    ("failing-by-assert-terse", [(Failure, "assertion failed")]),
    ("failing-by-assert-reason", [(Failure, "assert reason")]),
    ("failing-manually-terse", [(Failure, "test marked as failed")]),
    ("failing-manually-reason", [(Failure, "fail reason")]),
    ("error", [(Error, "")])
] == [(name, [(type(r), r.reason if hasattr(r, "reason") else "") for r in rez]) for name, rez in suite.results]

In [19]:
suite = Suite()

with suite.test("trial") as test:
    test.fail()
    
with suite.test("trial") as test:
    raise RuntimeError()
    
with suite.test("trial") as test:
    pass  # Literally!

assert [("trial", [Failure, Error, Success])] == [(name, [type(r) for r in rez]) for name, rez in suite.results]

In [124]:
import inspect
def _asdf():
    pass
filename = inspect.getfile(_asdf)


suite = Suite()

with suite.test("first") as test:
    test.fail()

with suite.test("first") as test:
    pass

with suite.test("second") as test:
    raise RuntimeError()

assert {name: [r["type"] for r in rez] for name, rez in suite.as_dict().items()} == {
    "first": ["Failure", "Success"],
    "second": ["Error"]
}

## Testing environment protection during test execution

In [20]:
assert "x" not in globals()

suite = Suite()
with suite.test("trial"):
    x = 5
    assert x == 5
    
assert "x" not in globals()
assert [("trial", [Success])] == [(name, [type(r) for r in rez]) for name, rez in suite.results]

In [21]:
assert "x" not in globals()

suite = Suite()
with suite.test("trial", protect_env=[]):  # Test this as [] has False boolean value.
    x = 5
    assert x == 5
    
assert "x" not in globals()
assert [("trial", [Success])] == [(name, [type(r) for r in rez]) for name, rez in suite.results]

In [22]:
assert "x" not in globals()
mylist = [1, 2, 3]

suite = Suite()
with suite.test("trial", protect_env=["mylist"]):
    x = 5
    assert x == 5
    mylist.append(4)
    
assert "x" not in globals()
assert [("trial", [Success])] == [(name, [type(r) for r in rez]) for name, rez in suite.results]
assert [1, 2, 3] == mylist